<a href="https://colab.research.google.com/github/g-zin/twitter-analysis-corona-virus/blob/master/web_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
from datetime import date
today= date.today()
d= today.strftime("%m-%d-%y")
print("date=",d)

date= 07-23-20


In [3]:
 cnn_url=" https://www.cnn.com/world/live-news/coronavirus-pandemic-{}-intl/index.html".format(d)


In [4]:
print(cnn_url)

 https://www.cnn.com/world/live-news/coronavirus-pandemic-07-23-20-intl/index.html


In [5]:
from bs4 import BeautifulSoup
import requests

In [6]:
html = requests.get(cnn_url).text

In [7]:
soup= BeautifulSoup(html)
print(soup.title)

<title data-rh="true">Coronavirus pandemic: Updates from around the world</title>


In [8]:
import spacy

nlp = spacy.load('en_core_web_sm')

In [9]:
for link in soup.find_all("h2"):
    print("Headline : {}".format(link.text))
    for ent in nlp(link.text).ents:
      print("\tText:{}, Entity : {}".format(ent.text,ent.label_))

Headline : What you need to know
Headline : You'll have to wear a face mask in these places in England starting tomorrow
	Text:England, Entity : GPE
	Text:tomorrow, Entity : DATE
Headline : There's a "fundamental agreement" between White House and Senate GOP on stimulus plan, official says
	Text:White House, Entity : ORG
	Text:Senate, Entity : ORG
	Text:GOP, Entity : ORG
Headline : Subway ridership is up 75% in New York City, mayor says
	Text:Subway, Entity : ORG
	Text:75%, Entity : PERCENT
	Text:New York City, Entity : GPE
Headline : North Carolina National Guard will wind down Covid-19 efforts this week
	Text:North Carolina National Guard , Entity : ORG
	Text:this week, Entity : DATE
Headline : What's in the GOP stimulus plan, according to people briefed on it
	Text:GOP, Entity : ORG
Headline : Iraq surpasses 100,000 total cases of Covid-19
	Text:Iraq, Entity : GPE
	Text:100,000, Entity : CARDINAL
Headline : City of Miami has issued 115 tickets for mask violations this week, mayor sa

In [10]:
nbc_url="https://www.nbcnews.com/health/coronavirus"
cnbc_rss_url="https://www.cnbc.com/id/10000108/device/rss/rss.html"

In [11]:
urls=[cnn_url,nbc_url,cnbc_rss_url]
formats=['html.parser','html.parser','xml']
tags=['h2','h2','description']
website=['CNN','NBC','CNBC']

In [12]:
crawl_len=0
for url in urls:
    print("Crawling web page ..........{}".format(url))
    response = requests.get(url)
    soup = BeautifulSoup(response.content,formats[crawl_len])

    for link in soup.find_all(tags[crawl_len]):

      if(len(link.text.split(" ")) > 4):
        print("Headline : {}".format(link.text))

        entities=[]
        for ent in nlp(link.text).ents: 
          print("\tText : {}, Entity : {}".format(ent.text, ent.label_)) 

    crawl_len=crawl_len+1

Crawling web page .......... https://www.cnn.com/world/live-news/coronavirus-pandemic-07-23-20-intl/index.html
Headline : What you need to know
Headline : You'll have to wear a face mask in these places in England starting tomorrow
	Text : England, Entity : GPE
	Text : tomorrow, Entity : DATE
Headline : There's a "fundamental agreement" between White House and Senate GOP on stimulus plan, official says
	Text : White House, Entity : ORG
	Text : Senate, Entity : ORG
	Text : GOP, Entity : ORG
Headline : Subway ridership is up 75% in New York City, mayor says
	Text : Subway, Entity : ORG
	Text : 75%, Entity : PERCENT
	Text : New York City, Entity : GPE
Headline : North Carolina National Guard will wind down Covid-19 efforts this week
	Text : North Carolina National Guard , Entity : ORG
	Text : this week, Entity : DATE
Headline : What's in the GOP stimulus plan, according to people briefed on it
	Text : GOP, Entity : ORG
Headline : Iraq surpasses 100,000 total cases of Covid-19
	Text : Iraq

In [13]:
crawl_len=0
news_dict=[]
for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.content,formats[crawl_len])

    for link in soup.find_all(tags[crawl_len]):

      if(len(link.text.split(" ")) > 4):

        entities=[]

        entities=[(ent.text, ent.label_) for ent in nlp(link.text).ents]

        news_dict.append({'website':website[crawl_len],'url': url,'headline':link.text,'entities':entities})
    
    crawl_len=crawl_len+1

In [14]:
news_dict

[{'entities': [],
  'headline': 'What you need to know',
  'url': ' https://www.cnn.com/world/live-news/coronavirus-pandemic-07-23-20-intl/index.html',
  'website': 'CNN'},
 {'entities': [('GOP', 'ORG'), ('flux', 'GPE'), ('White House', 'ORG')],
  'headline': 'Timeline for release of GOP relief proposal is in flux, White House negotiators say',
  'url': ' https://www.cnn.com/world/live-news/coronavirus-pandemic-07-23-20-intl/index.html',
  'website': 'CNN'},
 {'entities': [],
  'headline': 'Want to reopen schools safely? Community spread "must not be explosive," expert says.',
  'url': ' https://www.cnn.com/world/live-news/coronavirus-pandemic-07-23-20-intl/index.html',
  'website': 'CNN'},
 {'entities': [('England', 'GPE'), ('tomorrow', 'DATE')],
  'headline': "You'll have to wear a face mask in these places in England starting tomorrow",
  'url': ' https://www.cnn.com/world/live-news/coronavirus-pandemic-07-23-20-intl/index.html',
  'website': 'CNN'},
 {'entities': [('White House', '

In [15]:
import pandas as pd 

In [16]:
news_df = pd.DataFrame(news_dict)

In [17]:
pd.set_option('max_colwidth',800)
news_df.head(20)

,website,url,headline,entities
0,CNN,https://www.cnn.com/world/live-news/coronavirus-pandemic-07-23-20-intl/index.html,What you need to know,[]
1,CNN,https://www.cnn.com/world/live-news/coronavirus-pandemic-07-23-20-intl/index.html,"Timeline for release of GOP relief proposal is in flux, White House negotiators say","[(GOP, ORG), (flux, GPE), (White House, ORG)]"
2,CNN,https://www.cnn.com/world/live-news/coronavirus-pandemic-07-23-20-intl/index.html,"Want to reopen schools safely? Community spread ""must not be explosive,"" expert says.",[]
3,CNN,https://www.cnn.com/world/live-news/coronavirus-pandemic-07-23-20-intl/index.html,You'll have to wear a face mask in these places in England starting tomorrow,"[(England, GPE), (tomorrow, DATE)]"
4,CNN,https://www.cnn.com/world/live-news/coronavirus-pandemic-07-23-20-intl/index.html,"There's a ""fundamental agreement"" between White House and Senate GOP on stimulus plan, official says","[(White House, ORG), (Senate, ORG), (GOP, ORG)]"
5,CNN,https://www.cnn.com/world/live-news/coronavirus-pandemic-07-23-20-intl/index.html,"Subway ridership is up 75% in New York City, mayor says","[(Subway, ORG), (75%, PERCENT), (New York City, GPE)]"
6,CNN,https://www.cnn.com/world/live-news/coronavirus-pandemic-07-23-20-intl/index.html,North Carolina National Guard will wind down Covid-19 efforts this week,"[(North Carolina National Guard , ORG), (this week, DATE)]"
7,CNN,https://www.cnn.com/world/live-news/coronavirus-pandemic-07-23-20-intl/index.html,"What's in the GOP stimulus plan, according to people briefed on it","[(GOP, ORG)]"
8,CNN,https://www.cnn.com/world/live-news/coronavirus-pandemic-07-23-20-intl/index.html,"Iraq surpasses 100,000 total cases of Covid-19","[(Iraq, GPE), (100,000, CARDINAL)]"
9,CNN,https://www.cnn.com/world/live-news/coronavirus-pandemic-07-23-20-intl/index.html,"City of Miami has issued 115 tickets for mask violations this week, mayor says","[(Miami, GPE), (115, CARDINAL), (this week, DATE)]"


In [19]:
news_final_df = pd.concat([news_df[['website','url','headline']],news_df['entities'].apply(pd.Series)],axis=1)

In [20]:
news_final_df.head()

,website,url,headline,0,1,2,3,4,5,6
0,CNN,https://www.cnn.com/world/live-news/coronavirus-pandemic-07-23-20-intl/index.html,What you need to know,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CNN,https://www.cnn.com/world/live-news/coronavirus-pandemic-07-23-20-intl/index.html,"Timeline for release of GOP relief proposal is in flux, White House negotiators say","(GOP, ORG)","(flux, GPE)","(White House, ORG)",NaN,NaN,NaN,NaN
2,CNN,https://www.cnn.com/world/live-news/coronavirus-pandemic-07-23-20-intl/index.html,"Want to reopen schools safely? Community spread ""must not be explosive,"" expert says.",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CNN,https://www.cnn.com/world/live-news/coronavirus-pandemic-07-23-20-intl/index.html,You'll have to wear a face mask in these places in England starting tomorrow,"(England, GPE)","(tomorrow, DATE)",NaN,NaN,NaN,NaN,NaN
4,CNN,https://www.cnn.com/world/live-news/coronavirus-pandemic-07-23-20-intl/index.html,"There's a ""fundamental agreement"" between White House and Senate GOP on stimulus plan, official says","(White House, ORG)","(Senate, ORG)","(GOP, ORG)",NaN,NaN,NaN,NaN


In [21]:
news_df.to_csv("web_scrapping.csv")